## ML3 Notebook

### Principal Component Analysis (PCA)

We have a dataset matrix $\mathbf{X}\in \mathbb{R}^{N\times D}$ that consists of $N$ data points, each of dimensionality $D$. We want to reduce the dimensionality of our dataset, so it takes up less space, and is easier to visualise. 

We can achieve this by using a transformation matrix $\mathbf{W}\in \mathbb{R}^{D\times M}$. This transforms our data to a lower dimension using $ \mathbf{X_{D}} = \mathbf{X}\mathbf{W}$.

To determine how good this transformation is, we can attempt to reconstruct our data using $ \mathbf{\widetilde{X}} =  \mathbf{X_D}\mathbf{W^T}$. We can then compute reconstruction error.

The transformation that **minimises reconstruction error** in this scenario is the matrix that consists of the first $M$ **principal components** of $\mathbf{X}$. We will compute these for the iris dataset in this notebook by following the PCA algorithm:



In [ ]:
import numpy as np # Import numpy
X = np.load('iris_standardised.npy') # Load our standardised iris data

print(f'Our dataset is represented by a matrix X that is of shape {X.shape}')

### 1. Make sure your data is standarised.

This dataset has already been standardised!

### 2. Construct the covariance matrix $\mathbf{C} =\frac{1}{D}\mathbf{X^T}\mathbf{X}$

In [ ]:
D = 4 
C = (1/D)*X.T@X

### 3. Compute the eigenvalues/vectors of $\mathbf{C}$ ensuring that they are unit norm.

This can be done numerically using `np.linalg.eig`.

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(C)

print(f'The eigenvalues of C are {eigenvalues}')
print(f'The eigenvectors of C are \n {eigenvectors}')

We can see that the eigenvalues are arranged highest to lowest. Each column in the matrix above contains the  an eigenvector corresponding to each eigenvalue. These are already unit norm (thanks Python!). This means that the first (or 0th in Python) column of `eigenvectors` is the 1st principal component, the second is the second principal component etc.

### Constructing the $\mathbf{W}$ that minimises reconstruction error

If we want to reduce our dataset from 4D to 2D, then the $\mathbf{W}$ that minimises reconstruction error (and is therefore a good transformation) is given by the matrix containing the first two principal components.

In [ ]:
W = eigenvectors[:,0:2] # Python notation takes some getting used to, but this is grabbing the first two columns of W
print(W)

Let's use this to compute reconstruction error, and confirm that it is low.

In [ ]:
X_D = X@W # Use W to reduce X down to XD
X_tilde = X@W@W.T # Use W to reduce X down, and then W.T to bring it back up to attempt reconstruction
E = np.sum(np.sum((X-X_tilde)**2,1)**.5) # Compute reconstruction error
print(E)

This is several times lower than our naive choice of $\mathbf{W}$ but is non-zero. That's because **in most cases we can't have perfect reconstruction if we throw away information**. It's a compromise. 

Let's see what happens if we instead go from 4D to 3D, using the $\mathbf{W}$ that minimises reconstruction error.

In [ ]:
W = eigenvectors[:,0:3] # Get the first three principal components
print(W)
X_D_3D = X@W # Use W to reduce X down to XD
X_tilde = X@W@W.T # Use W to reduce X down, and then W.T to bring it back up to attempt reconstruction
E = np.sum(np.sum((X-X_tilde)**2,1)**.5) # Compute reconstruction error
print(E)

This is a lower error, because we are throwing away less information by going down to 3D. 

### Visualising our data

So what does our data look like? We can't plot data in 4 dimensions. Let's just take a look at the first two. We will also colour each point to correspond to the type of flower it has been labelled as. **Each flower in the dataset has been labelled as either setosa, virginica and versicolor.**

Let's plot the first two dimensions of our **original standardised data** (sepal length, sepal width). Note that these are not the same as the two new dimensions that PCA gives us.


In [ ]:
import matplotlib.pyplot as plt # Import matplotlib for plotting
import sklearn.datasets # If you are running this locally, then `pip install sklearn` in your Python environment.
iris = sklearn.datasets.load_iris() #Get the flower labels from sklearn.


species0 = iris.target == 0 # Find flowers labelled as class 0 
species1 = iris.target == 1 # Find flowers labelled as class 1 
species2 = iris.target == 2 # Find flowers labelled as class 2 

plt.scatter(X[species0,0],X[species0,1] ,color='b') # Plot standardised data in class 0 in blue
plt.scatter(X[species1,0],X[species1,1], color='g') # Plot standardised data in class 1 in green
plt.scatter(X[species2,0],X[species2,1], color='r') # Plot standardised data in class 2 in red

plt.xlabel('sepal length (standardised)') # Label your axes!
plt.ylabel('sepal width (standardised)') # Label your axes!
    
plt.legend(iris.target_names)  # Put class names in the legend
    


It's quite difficult in this space to be able to tell veriscolor, and virginica apart. This will be difficult for classification (which we will learn about soon). Let's see what our space looks like once we've used $\mathbf{W}$ to take the original data down to 2D:

In [ ]:
plt.scatter(X_D[species0,0],X_D[species0,1] ,color='b') # Plot PCA projected data in class 0 in blue
plt.scatter(X_D[species1,0],X_D[species1,1], color='g') # Plot PCA projected data in class 1 in green
plt.scatter(X_D[species2,0],X_D[species2,1], color='r') # Plot PCA projected data in class 2 in red

plt.xlabel('PCA dimension 1') # Label your axes!
plt.ylabel('PCA dimension 2') # Label your axes!
    
plt.legend(iris.target_names)  # Put class names in the legend

The three types of flowers are now more distinct! By using a transformation to minimise reconstruction error, the data lies in a space where these is maximal **variation**. In other words, what PCA has done is found the best linear combination of the original 4 dimensions such that data is varied as much as possible in 2 dimensions

Let's finish by viewing our data reduced to 3D using PCA.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_D_3D[species0,0],X_D_3D[species0,1],X_D_3D[species0,2] ,color='b') # Plot PCA projected data in class 0 in blue
ax.scatter(X_D_3D[species1,0],X_D_3D[species1,1],X_D_3D[species0,2], color='g') # Plot PCA projected data in class 1 in green
ax.scatter(X_D_3D[species2,0],X_D_3D[species2,1],X_D_3D[species0,2], color='r') # Plot PCA projected data in class 2 in red

ax.set_xlabel('PCA dimension 1') # Label your axes!
ax.set_ylabel('PCA dimension 2') # Label your axes!
ax.set_zlabel('PCA dimension 3') # Label your axes!

plt.legend(iris.target_names)

